<a href="https://colab.research.google.com/github/aditichadha1310/CS4225/blob/main/cs4225_a2_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.1-bin-hadoop3.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [43]:
# After downloading dataset, you should have a file "network.csv" in your Files (click the folder icon in the left sidebar)
!wget https://bhooi.github.io/teaching/cs4225/bomb/part2/network.csv

--2022-11-05 10:22:41--  https://bhooi.github.io/teaching/cs4225/bomb/part2/network.csv
Resolving bhooi.github.io (bhooi.github.io)... 185.199.108.153, 185.199.111.153, 185.199.109.153, ...
Connecting to bhooi.github.io (bhooi.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 897278 (876K) [text/csv]
Saving to: ‘network.csv.1’

network.csv.1       100%[===================>] 876.25K  --.-KB/s    in 0.05s   

2022-11-05 10:22:42 (18.2 MB/s) - ‘network.csv.1’ saved [897278/897278]



In [44]:
df = spark.read.option('header', True).csv("network.csv").repartition(1).cache()
df.toPandas()

,src,dst,day
0,172.016.117.052,207.025.071.144,0
1,230.001.010.020,172.016.112.050,0
2,209.074.060.168,172.016.113.050,0
3,010.020.030.040,172.016.112.050,0
4,230.001.010.020,172.016.112.050,0
...,...,...,...
25877,208.239.005.230,192.168.001.001,29
25878,199.174.194.123,172.016.114.050,29
25879,166.102.114.043,172.016.113.050,29
25880,172.016.116.201,207.046.130.168,29


In [87]:
lines_wo_duplicates = df.dropDuplicates(["src", "dst"])
not_registered_users = lines_wo_duplicates.filter(lines_wo_duplicates.day % 7 != 0)
verified_users = lines_wo_duplicates.subtract(not_registered_users)
lines_with_count = verified_users.groupBy("dst").count()
final_dst = lines_with_count.filter("count == 13")
final_dst.write.format("csv").save("solutions")
final_dst.toPandas()


# Write your code for processing the data here.
# Colab tips: 
# Check Tools > Keyboard shortcuts. Some useful ones: 
# press Ctrl+Enter to run a cell, or Shift+Enter to run and go to next cell
# Ctrl+M then b to create a new cell after the current
# Ctrl+M then a to create a new cell before the current
# Ctrl+M then d deletes the current cell
# You can use multi-cursor / multi-selection in Colab (check the shortcuts in Tools)
# If you leave colab idle for too long, it may disconnect. The local memory may or may not be preserved, depending on how long you wait.
# To quickly view a DataFrame, you can use .toPandas(). 
# If you want to view it in full, write it to a file: df.write.format("csv").save("..."). 
# Then it should be viewable in your Files (click the folder icon in the left sidebar). You can double click it to view or download to view locally.
# If you want it to appear as 1 file, use df.repartition(1). 

,dst,count
0,207.025.071.025,13


In [ ]:
# TO CHECK YOUR ANSWERS: run "python login.py" (or python3 if necessary to run python 3. This should be run locally on your computer).

# TO SUBMIT YOUR ANSWERS: submit them to LumiNUS > Quiz > Assignment 2. Then paste your code into submission.txt (locally on your computer) and upload it to the LumiNUS submission folder.